In [48]:
import pandas as pd
import numpy as np
import re
import time
import random
import gensim
import jieba
from numpy import *
import json
from numpy import linalg as la  

In [169]:
wu=pd.read_csv('wvusermatrix.csv',encoding='gbk')#word2vec用户矩阵
wt=pd.read_csv('wvtvmatrix2.csv',encoding='gbk')#word2vec节目矩阵
tu=pd.read_csv('tagusermatrix.csv',encoding='gbk')#标签用户矩阵
cu=pd.read_csv('channelusermatrix.csv',encoding='gbk')#标签用户频道矩阵
tt=pd.read_csv('tagtvmatrix2.csv',encoding='gbk')#标签节目矩阵
dfr=pd.read_csv('mingzilianhe.csv',encoding='gbk')#用户看过的节目
info=pd.read_csv('info3.csv',encoding='gbk').fillna('')#节目信息


for i in range(len(tu)):
    if np.sum(tu.loc[i,:])!=0:
        tu.loc[i,:]=tu.loc[i,:]/np.sum(tu.loc[i,:])
for i in range(len(cu)):
    if np.sum(cu.loc[i,:])!=0:
        cu.loc[i,:]=cu.loc[i,:]/np.sum(cu.loc[i,:])
tu=tu+cu*0.2
tu.loc[:,'新闻']=0.0
typelist=['犯罪', '喜剧', '舞台艺术', '新闻', '儿童', '真人秀', '歌舞', '戏曲', '音乐', '冒险', '灾难', '恐怖', '家庭', '短片', '历史', '运动', '动画', '纪录片', '惊悚', '科幻', '战争', '动作', '健康', '脱口秀', '悬疑', '剧情', '古装', '鬼怪', '奇幻', '武侠', '同性', '体育', '西部', '传记', '爱情']
typelist=sorted(typelist)

In [51]:
count=0
nsum=0
ratelist=[]
for i in range(len(info)):
    if info.loc[i,'rate']=='':
        continue
    di=json.loads(info.loc[i,'rate'])
    if di["average"]>0:
        ratelist.append(di["average"])
        nsum+=di["average"]
        count+=1
print(nsum/count)
print(sorted(ratelist)[len(ratelist)//2])

6.690473764645957
6.9


In [69]:
rate={}
for i in range(len(info)):
    if info.loc[i,'rate']=='':
        rate[info.loc[i,'name']]=6.69
        continue
    di=json.loads(info.loc[i,'rate'])
    if di["average"]>0:
        rate[info.loc[i,'name']]=di["average"]
    else:
        rate[info.loc[i,'name']]=6.69

In [3]:
#求余弦
def cosSim(inA,inB):
    num = float(np.sum(inA*inB))
    denom = la.norm(inA)*la.norm(inB)
    if denom==0.0:
        return 0.0
    return num/denom

In [70]:
def getRec(u,wu=wu,wt=wt,tu=tu,tt=tt,dfr=dfr,info=info):
    print(u,dfr.loc[u,'name'])
    ctlist=[]#记录出现过的标签
    if np.sum(tu.loc[u,:])==0:
        print('该用户无历史数据')
        return []
    ua1=np.array(wu.loc[u,:])
    ua2=np.array(tu.loc[u,:])
    tv=[]
    for i in range(len(wt)):
        tv.append([i,cosSim(ua1,np.array(wt.loc[i,:]))*0.4])
    for i in range(len(tt)):
        tv[i][1]+=cosSim(ua2,np.array(tt.loc[i,:]))*0.6-0.01+0.01*rate[info.loc[i,'name']]
    tv=sorted(tv,key=lambda k:k[1],reverse=True)
    for t in tv[:12]:
        print(info.loc[t[0],'name'],info.loc[t[0],'info'],t[1])
    

In [147]:
def getRec(u,wu=wu,wt=wt,tu=tu,tt=tt,dfr=dfr,info=info):
    print(u)
    print(dfr.loc[u,'name'])
    ctlist=[]#记录出现过的标签
    if np.sum(tu.loc[u,:])==0:
        print('该用户无历史数据')
        return []
    ua1=np.array(wu.loc[u,:])
    ua2=np.array(tu.loc[u,:])
    tv=[]
    for i in range(len(wt)):
        tv.append([i,cosSim(ua1,np.array(wt.loc[i,:]))*0.2])
    for i in range(len(tt)):
        tv[i][1]+=cosSim(ua2,np.array(tt.loc[i,:]))*0.7+0.01*rate[info.loc[i,'name']]
    tv=sorted(tv,key=lambda k:k[1],reverse=True)
    for t in tv[:10]:
        ctlist+=json.loads(info.loc[t[0],'info'])
        print(info.loc[t[0],'name'],info.loc[t[0],'info'],t[1])
    #第二轮推荐
    ctlist=list(set(ctlist))
    now=pd.DataFrame(np.ones((1,35)),columns=typelist)
    for c in ctlist:
        now.loc[0,c]=0.5
    ua2=np.array(tu.loc[u,:])*np.array(now.loc[0,:])
    tv=[]
    for i in range(len(wt)):
        tv.append([i,cosSim(ua1,np.array(wt.loc[i,:]))*0.2])
    for i in range(len(tt)):
        tv[i][1]+=cosSim(ua2,np.array(tt.loc[i,:]))*0.7+0.01*rate[info.loc[i,'name']]
    tv=sorted(tv,key=lambda k:k[1],reverse=True)
    for t in tv[:10]:
        print(info.loc[t[0],'name'],info.loc[t[0],'info'],t[1])

In [148]:
getRec(265)

265
 楚乔传 楚乔传 楚乔传 我的一九九七 楚乔传 楚乔传 楚乔传 大军师司马懿之军师联盟 我的一九九七 踩过界 踩过界 楚乔传 我的前半生 宿醉1 楚乔传 我的前半生 我的前半生 我的前半生 我的前半生 宿醉1 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的一九九七 我的一九九七 我的一九九七 我的前半生 赌城群英会 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 谋杀似水年华 谋杀似水年华 谋杀似水年华 我的一九九七 谋杀似水年华 大军师司马懿之军师联盟 绣春刀 栀子花开 栀子花开 绣春刀 赛琳娜 赛琳娜 贱女孩 深夜食堂 贱女孩 醉玲珑 故事中的故事 天天向上 故事中的故事 我的前半生 我的前半生 反恐特战队之猎影 我的前半生 我的前半生 反恐特战队之猎影 我的前半生 反恐特战队之猎影 我的前半生 大军师司马懿之军师联盟 大军师司马懿之军师联盟 我的前半生 反恐特战队之猎影 反恐特战队之猎影 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 我的前半生 中国新歌声 中国新歌声 中国新歌声 我们所知道的生活 我们所知道的生活 我们所知道的生活 中国新歌声 迷色 我们所知道的生活 迷色 冒牌家庭 迷色 秦时丽人明月心 迷色 冒牌家庭 中国新歌声 铁道飞虎 源代码 源代码 超凡蜘蛛侠2 铁道飞虎 超凡蜘蛛侠2 超凡蜘蛛侠 源代码 中国新歌声 那年花开月正圆 那年花开月正圆 那年花开月正圆 秦时丽人明月心 那年花开月正圆 中国新歌声 中国新歌声 她 她 前目的地 她 她 前目的地 她 敢死队3 她 敢死队3 一生一世 一生一世 猩球崛起 猩球崛起 猩球崛起 猩球崛起  庆祝香港回归二十周年—两岸青年大联欢 奔跑吧挑战者联盟挑战者联盟奔跑吧跑男来了天气预报今日聚焦新闻深一度西湖人文深呼吸江南·戏文传奇华少爱读书南宋记住乡愁记住乡愁新闻深一度浙江新闻联播王牌对王牌来吧冠军高能少年团 快乐大本营 转播中央台新闻联播  快乐大本营 快乐大

In [177]:
np.max(tu.loc[0,:])

0.20350623069549473

In [188]:
tu.loc[0,:].sort_values(axis=0,ascending=False)[:10].keys().tolist()

['动画', '家庭', '剧情', '喜剧', '纪录片', '奇幻', '真人秀', '科幻', '动作', '爱情']

In [193]:
tu.loc[0,:].sort_values(axis=0,ascending=False)[:10].as_matrix().tolist()

[0.20350623069549473,
 0.18257417071987314,
 0.10925125146447971,
 0.08476302055596975,
 0.08334859942485888,
 0.07701778677175418,
 0.06935740405438989,
 0.06320587922036426,
 0.061484716157205244,
 0.05581768263097478]

In [199]:
utt=pd.DataFrame(columns=['用户','tag1','tag2','推荐指数'])

In [202]:
n=0
for i in range(len(tu)):
    mm=tu.loc[i,:].max()
    if mm==0:
        continue
    ming=tu.loc[i,:].sort_values(axis=0,ascending=False)[:10].keys().tolist()
    zhi=tu.loc[i,:].sort_values(axis=0,ascending=False)[:10].as_matrix().tolist()
    for j in range(10):
        if zhi[j]==0:
            break
        utt.loc[n,'用户']=i+10001
        utt.loc[n,'tag1']='收视偏好'
        utt.loc[n,'tag2']=ming[j]
        utt.loc[n,'推荐指数']=zhi[j]/mm
        n+=1

In [203]:
utt

,用户,tag1,tag2,推荐指数
0,10001,收视偏好,动画,1
1,10001,收视偏好,家庭,0.897143
2,10001,收视偏好,剧情,0.536845
3,10001,收视偏好,喜剧,0.416513
4,10001,收视偏好,纪录片,0.409563
5,10001,收视偏好,奇幻,0.378454
6,10001,收视偏好,真人秀,0.340812
7,10001,收视偏好,科幻,0.310584
8,10001,收视偏好,动作,0.302127
9,10001,收视偏好,爱情,0.27428


In [204]:
utt.to_csv('result/201800010问题二推荐结果表.csv',index=False)

In [208]:
uttt=pd.DataFrame(columns=['用户','tag1','tag2','tag3'])
n=0
for i in range(len(tu)):
    if tu.loc[i,:].sum()==0:
        continue
    mm=tu.loc[i,:].argmax()
    uttt.loc[n,'用户']=i+10001
    uttt.loc[n,'tag1']='收视偏好'
    uttt.loc[n,'tag2']='类别'
    uttt.loc[n,'tag3']=mm
    n+=1

In [209]:
uttt

,用户,tag1,tag2,tag3
0,10001,收视偏好,类别,动画
1,10002,收视偏好,类别,真人秀
2,10003,收视偏好,类别,剧情
3,10004,收视偏好,类别,真人秀
4,10005,收视偏好,类别,真人秀
5,10008,收视偏好,类别,真人秀
6,10009,收视偏好,类别,体育
7,10010,收视偏好,类别,爱情
8,10011,收视偏好,类别,儿童
9,10012,收视偏好,类别,体育


In [210]:
ut1=pd.read_csv('result/timetag.csv',encoding='gbk')

In [211]:
ut2=pd.read_csv('result/家庭成员.csv',encoding='gbk')

In [214]:
ut1.columns=['用户','tag1','tag2','tag3']

In [216]:
uu=pd.concat([uttt,ut1,ut2])

In [220]:
uu=uu.sort_values(by=['用户']).reset_index(drop=True)

In [222]:
uu.to_csv('result/201800010问题二用户数据标签标表.csv',index=False)

In [224]:
cp=pd.DataFrame(columns=['name','tag1','tag2','tag3'])
for i in range(len(tt)):
    cp.loc[i,'name']=info.loc[i,'name']
    cp.loc[i,'tag1']='基本特征'
    cp.loc[i,'tag2']='类别'
    cp.loc[i,'tag3']=tt.loc[i,:].argmax()

In [226]:
cp.to_csv('result/201800010问题二产品数据标签表.csv',index=False)